In [1]:
import pandas as pd, json

from sqlalchemy import create_engine
from config import config

<h1>Modelling Automation</h1>
<br>
<div>As the training of the models requires a coordinated effort, this workbook supports in splitting the supervision of the training among members of the project team by assigning the workload evenly. The output is "assignmen.json"</div>

In [2]:
config=config()
engine=create_engine("postgresql://"+config["user"]+":"+config["password"]+"@"+config["host"]+"/"+config["database"])

In [3]:
sql=("select trips.line_id,trips.direction,count(*) "
"from trips, leavetimes as lt "
"where trips.daystamp=lt.daystamp and trips.trip_id=lt.trip_id "
"group by trips.line_id,trips.direction;")

In [4]:
df = pd.read_sql(sql,engine)

In [6]:
sorted_lines=df.sort_values(by=["count"],ascending=False,ignore_index=True)

In [7]:
sorted_lines.to_csv("line_counts.csv")

In [8]:
def create_snake(index_length,snake_length):
    """creates a list of indexes from 0 to (snake_length-1) in snake order"""
    snake_list=[]
    for i in range(index_length):
        if i//(snake_length)%2==0:
            index=i%snake_length
        else:
            index=snake_length-1-i%snake_length
        snake_list.append(index)
    return snake_list

In [9]:
modellers=["yuqian","callum","jakob"]

In [10]:
def assign_models(sorted_lines,modellers):
    model_count=sorted_lines.shape[0]
    assignment={}
    for modeller in modellers:
        assignment[modeller]=[]
    snake=create_snake(model_count,len(modellers))
    for i in range(model_count):
        modeller=modellers[snake[i]]
        row=sorted_lines.iloc[i]
        assignment[modeller].append([str(row["line_id"]),int(row["direction"]),int(row["count"])])
    return assignment

In [11]:
assignment=assign_models(sorted_lines,modellers)

In [12]:
assignment

{'yuqian': [['40', 2, 2259400],
  ['145', 2, 2099725],
  ['27', 2, 2055488],
  ['39A', 1, 1838332],
  ['16', 1, 1717429],
  ['14', 1, 1356188],
  ['9', 1, 1355742],
  ['83', 1, 1090619],
  ['4', 2, 1071897],
  ['4', 1, 1009559],
  ['130', 1, 998538],
  ['140', 2, 910603],
  ['122', 1, 909464],
  ['39', 1, 831487],
  ['15B', 2, 831121],
  ['41C', 1, 799659],
  ['67', 1, 796387],
  ['33', 2, 760814],
  ['29A', 2, 760409],
  ['27B', 1, 728724],
  ['33', 1, 724845],
  ['1', 2, 686132],
  ['66', 1, 665855],
  ['31', 2, 634489],
  ['31', 1, 630196],
  ['66', 2, 594695],
  ['17', 2, 589061],
  ['38', 1, 563222],
  ['25A', 1, 561428],
  ['102', 2, 511631],
  ['84', 2, 509373],
  ['44', 2, 463186],
  ['44', 1, 457529],
  ['120', 1, 418295],
  ['68', 2, 414013],
  ['76', 2, 396509],
  ['43', 2, 395860],
  ['65B', 1, 385365],
  ['38A', 1, 385119],
  ['65B', 2, 347538],
  ['79', 2, 345112],
  ['47', 2, 323175],
  ['47', 1, 320740],
  ['79A', 1, 292166],
  ['61', 2, 290666],
  ['66B', 1, 260003],
 

In [13]:
for modeller in assignment:
    sum_rows=0
    for row in assignment[modeller]:
        sum_rows+=row[2]
    print(f"{modeller} trains {sum_rows} rows.")

yuqian trains 39028084 rows.
callum trains 38968090 rows.
jakob trains 38952939 rows.


In [14]:
with open('assignment.json', 'w') as json_file:
  json.dump(assignment, json_file)